In [2]:
import numpy as np
import pandas as pd
import mne
import os
import matplotlib.pyplot as plt
from MNE_Pipeline import MNE_Repo_Mat as MP
from multiprocessing import Process
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from matplotlib import cm
import sys
import cv2
import re

In [4]:
train_s = 'topomaps_RT_100/train/combined'
test_s = 'topomaps_RT_100/test/combined'
labels = [0, 1, 3]
total_t = 0
t_total = 0
for s in labels:
    path_t = os.path.join(train_s, str(s))
    total_t = total_t + len(os.listdir(path_t))
    t_path = os.path.join(test_s, str(s))
    t_total = t_total + len(os.listdir(t_path))
    
total_t + t_total

53144

In [3]:
mp = MP()
info = mp.construct_info()

/home/kam/anaconda3/envs/mne/lib/python3.7/site-packages/mne/utils/docs.py:830: DeprecationWarning: Function read_montage is deprecated; ``read_montage`` is deprecated and will be removed in v0.20. Please use ``read_dig_fif``, ``read_dig_egi``, ``read_custom_montage``, or ``read_dig_captrack`` to read a digitization based on your needs instead; or ``make_standard_montage`` to create ``DigMontage`` based on template; or ``make_dig_montage`` to create a ``DigMontage`` out of np.arrays
  warnings.warn(msg, category=DeprecationWarning)
/home/kam/anaconda3/envs/mne/lib/python3.7/site-packages/mne/utils/docs.py:813: DeprecationWarning: Class Montage is deprecated; Montage class is deprecated and will be removed in v0.20. Please use DigMontage instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/kam/dl_cp_gen_topomap/MNE_Pipeline.py:57: DeprecationWarning: Setting a montage using anything rather than DigMontage is deprecated and will raise an error in v0.20. Please use ``read_dig_

In [4]:
def combine_topomaps(alpha_path, beta_path, gamma_path, _class, folder_path):
    full_path_w_class = os.path.join(folder_path, str(_class))
    
    if not os.path.exists(full_path_w_class):
        os.makedirs(full_path_w_class)
        
    alpha_path = os.path.join(alpha_path, str(_class))
    beta_path = os.path.join(beta_path, str(_class))
    gamma_path = os.path.join(gamma_path, str(_class))
    
        
    alpha_imgs = [os.path.join(alpha_path, img_id) for img_id in os.listdir(alpha_path)]
    beta_imgs = [os.path.join(beta_path, img_id) for img_id in os.listdir(beta_path)]
    gamma_imgs = [os.path.join(gamma_path, img_id) for img_id in os.listdir(gamma_path)]
    
    trials = os.listdir(alpha_path)
    
    for path_alpha, path_beta, path_gamma, trial_no in tqdm(zip(alpha_imgs, beta_imgs, gamma_imgs, trials)):
        
        trial_no = int(re.search(r'\d+', trial_no).group())
        
        img_alpha = cv2.imread(path_alpha, cv2.IMREAD_GRAYSCALE)
        img_beta = cv2.imread(path_beta, cv2.IMREAD_GRAYSCALE)
        img_gamma = cv2.imread(path_gamma, cv2.IMREAD_GRAYSCALE)
        
        combined_img = np.dstack((img_alpha, img_beta, img_gamma))
        
        save_path = os.path.join(full_path_w_class, 'bts_{}.png'.format(trial_no))
        
        cv2.imwrite(save_path, combined_img)
    

def gen_topomap(avg_band_pw_data, labels, classes, trial_names, folder_path, start, end):
    
    for _class in classes:
        full_path_w_class = os.path.join(folder_path, str(_class))
        if not os.path.exists(full_path_w_class):
            os.makedirs(full_path_w_class)
    
    for i in range(start, end):
        save_path = os.path.join(folder_path, str(labels[i]), trial_names[i])
        topo , _ = mne.viz.plot_topomap(avg_band_pw_data[i], 
                                        info, res=256, show=False, contours=0)

        topo.get_figure().savefig(save_path, dpi=64)
    

def async_gen_topomap(x, y, classes, trial_names, start, end, folder, t='train'):
    
    alpha = x[:, 0:64]
    beta = x[:, 64:128]
    gamma = x[:, 128:192]
    
    n = list(range(start, end, 50))
    n.append(end)
    processes = []
    for i in range(len(n) - 1):
        process_alpha = Process(target=gen_topomap, 
                                args=(alpha, y, classes, trial_names, os.path.join(folder, t, 'alpha'), n[i], n[i+1]))
        process_beta = Process(target=gen_topomap, 
                               args=(beta, y, classes, trial_names, os.path.join(folder, t, 'beta'), n[i], n[i+1]))
        process_gamma = Process(target=gen_topomap, 
                                args=(gamma, y, classes, trial_names, os.path.join(folder, t, 'gamma'), n[i], n[i+1]))

        process_alpha.start()
        process_beta.start()
        process_gamma.start()


        processes.extend([process_alpha, process_beta, process_gamma])
        
    for process in processes:
        process.join()

## Topomap Generation 1vs3

In [3]:
data = pd.read_csv('data_1vs3/data_1vs3.csv', header=None).values
labels = pd.read_csv('data_1vs3/labels_1vs3.csv', header=None).values

scaler = MinMaxScaler()

X = scaler.fit_transform(data)
Y = np.where(labels == 0, 1, 3)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=33, test_size=0.2)

In [6]:
indices = list(range(0, len(x_train), 1000))

indices.append(len(x_train))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_train, y_train, indices[i], indices[i+1])

In [7]:
indices = list(range(0, len(x_test), 1000))

indices.append(len(x_test))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_test, y_test, indices[i], indices[i+1], t='test')

In [21]:
len(os.listdir('topomaps/train/combined/3'))

19989

In [18]:
combine_topomaps('topomaps/train/alpha', 'topomaps/train/beta', 'topomaps/train/gamma', 1, 'topomaps/train/combined')

In [19]:
combine_topomaps('topomaps/train/alpha', 'topomaps/train/beta', 'topomaps/train/gamma', 3, 'topomaps/train/combined')

In [22]:
combine_topomaps('topomaps/test/alpha', 'topomaps/test/beta', 'topomaps/test/gamma', 1, 'topomaps/test/combined')

In [23]:
combine_topomaps('topomaps/test/alpha', 'topomaps/test/beta', 'topomaps/test/gamma', 3, 'topomaps/test/combined')

## Topomap Generation 1 and 5 vs 3

In [32]:
data = pd.read_csv('data_15vs3/data.csv', header=None).values
labels = pd.read_csv('data_15vs3/labels.csv', header=None).values

scaler = MinMaxScaler()

X = scaler.fit_transform(data)
Y = np.where(labels == 0, 3, 15)

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=33, test_size=0.2)

In [37]:
indices = list(range(0, len(x_train), 1000))

indices.append(len(x_train))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_train, y_train, [3,15], indices[i], indices[i+1], 'topomaps_15vs3')

In [38]:
indices = list(range(0, len(x_test), 1000))

indices.append(len(x_test))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_test, y_test, [3,15], indices[i], indices[i+1], 'topomaps_15vs3', t='test')

In [39]:
combine_topomaps('topomaps_15vs3/train/alpha', 'topomaps_15vs3/train/beta', 'topomaps_15vs3/train/gamma', 3, 'topomaps_15vs3/train/combined')
combine_topomaps('topomaps_15vs3/train/alpha', 'topomaps_15vs3/train/beta', 'topomaps_15vs3/train/gamma', 15, 'topomaps_15vs3/train/combined')

In [40]:
combine_topomaps('topomaps_15vs3/test/alpha', 'topomaps_15vs3/test/beta', 'topomaps_15vs3/test/gamma', 3, 'topomaps_15vs3/test/combined')
combine_topomaps('topomaps_15vs3/test/alpha', 'topomaps_15vs3/test/beta', 'topomaps_15vs3/test/gamma', 15, 'topomaps_15vs3/test/combined')

## Topomap Generation 1 vs 5

In [46]:
data = pd.read_csv('data_1vs5/data_1vs5.csv', header=None).values
labels = pd.read_csv('data_1vs5/labels_1vs5.csv', header=None).values

scaler = MinMaxScaler()

X = scaler.fit_transform(data)
Y = np.where(labels == 0, 1, 5)

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=33, test_size=0.2)

In [47]:
indices = list(range(0, len(x_train), 1000))

indices.append(len(x_train))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_train, y_train, [1,5], indices[i], indices[i+1], 'topomaps_1vs5')

In [48]:
indices = list(range(0, len(x_test), 1000))

indices.append(len(x_test))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_test, y_test, [1,5], indices[i], indices[i+1], 'topomaps_1vs5', t='test')

In [49]:
combine_topomaps('topomaps_1vs5/train/alpha', 'topomaps_1vs5/train/beta', 'topomaps_1vs5/train/gamma', 1, 'topomaps_1vs5/train/combined')
combine_topomaps('topomaps_1vs5/train/alpha', 'topomaps_1vs5/train/beta', 'topomaps_1vs5/train/gamma', 5, 'topomaps_1vs5/train/combined')

In [50]:
combine_topomaps('topomaps_1vs5/test/alpha', 'topomaps_1vs5/test/beta', 'topomaps_1vs5/test/gamma', 1, 'topomaps_1vs5/test/combined')
combine_topomaps('topomaps_1vs5/test/alpha', 'topomaps_1vs5/test/beta', 'topomaps_1vs5/test/gamma', 5, 'topomaps_1vs5/test/combined')

## Topomap Generation RT

In [4]:
data = pd.read_csv('data_RT/data_bp_RT.csv', header=None).values
labels = pd.read_csv('data_RT/labels_RT.csv', header=None).values

scaler = MinMaxScaler()

X = scaler.fit_transform(data)
Y = labels.astype(np.int16)

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=33, test_size=0.2)

In [14]:
data.shape

(70505, 192)

In [5]:
indices = list(range(0, len(x_train), 1000))

indices.append(len(x_train))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_train, y_train, [0,1,3], indices[i], indices[i+1], 'topomaps_RT', t='train')

In [6]:
indices = list(range(0, len(x_test), 1000))

indices.append(len(x_test))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_test, y_test, [0, 1, 3], indices[i], indices[i+1], 'topomaps_RT', t='test')

In [15]:
combine_topomaps('topomaps_RT/train/alpha', 'topomaps_RT/train/beta', 'topomaps_RT/train/gamma', 0, 'topomaps_RT/train/combined')
combine_topomaps('topomaps_RT/train/alpha', 'topomaps_RT/train/beta', 'topomaps_RT/train/gamma', 1, 'topomaps_RT/train/combined')
combine_topomaps('topomaps_RT/train/alpha', 'topomaps_RT/train/beta', 'topomaps_RT/train/gamma', 3, 'topomaps_RT/train/combined')

In [16]:
combine_topomaps('topomaps_RT/test/alpha', 'topomaps_RT/test/beta', 'topomaps_RT/test/gamma', 0, 'topomaps_RT/test/combined')
combine_topomaps('topomaps_RT/test/alpha', 'topomaps_RT/test/beta', 'topomaps_RT/test/gamma', 1, 'topomaps_RT/test/combined')
combine_topomaps('topomaps_RT/test/alpha', 'topomaps_RT/test/beta', 'topomaps_RT/test/gamma', 3, 'topomaps_RT/test/combined')

In [18]:
16787+20007+19610

56404

## Topomap Generation RT (Rate-100)

In [4]:
data = pd.read_csv('data_RT_100/data_bp_RT_100.csv', header=None).values
labels = pd.read_csv('data_RT_100/labels_RT_100.csv', header=None).values

scaler = MinMaxScaler()

X = scaler.fit_transform(data)
Y = labels.astype(np.int16)

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=33, test_size=0.2)

In [5]:
x_train.shape

(42515, 192)

In [6]:
#Train
indices = list(range(0, len(x_train), 1000))

indices.append(len(x_train))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_train, y_train, [0,1,3], indices[i], indices[i+1], 'topomaps_RT_100', t='train')
    
#test
indices = list(range(0, len(x_test), 1000))

indices.append(len(x_test))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_test, y_test, [0, 1, 3], indices[i], indices[i+1], 'topomaps_RT_100', t='test')
    
# Combine Train

combine_topomaps('topomaps_RT_100/train/alpha', 'topomaps_RT_100/train/beta', 'topomaps_RT_100/train/gamma', 0, 'topomaps_RT_100/train/combined')
combine_topomaps('topomaps_RT_100/train/alpha', 'topomaps_RT_100/train/beta', 'topomaps_RT_100/train/gamma', 1, 'topomaps_RT_100/train/combined')
combine_topomaps('topomaps_RT_100/train/alpha', 'topomaps_RT_100/train/beta', 'topomaps_RT_100/train/gamma', 3, 'topomaps_RT_100/train/combined')

# Combine Test

combine_topomaps('topomaps_RT_100/test/alpha', 'topomaps_RT_100/test/beta', 'topomaps_RT_100/test/gamma', 0, 'topomaps_RT_100/test/combined')
combine_topomaps('topomaps_RT_100/test/alpha', 'topomaps_RT_100/test/beta', 'topomaps_RT_100/test/gamma', 1, 'topomaps_RT_100/test/combined')
combine_topomaps('topomaps_RT_100/test/alpha', 'topomaps_RT_100/test/beta', 'topomaps_RT_100/test/gamma', 3, 'topomaps_RT_100/test/combined')

## Topomap Generation Triggs - 30

In [5]:
data = pd.read_csv('data_triggs_30/data_bp_all_triggs_30.csv', header=None).values
labels = pd.read_csv('data_triggs_30/labels_all_triggs_30.csv', header=None).values

scaler = MinMaxScaler()

X = scaler.fit_transform(data)
Y = labels.astype(np.int16)

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=33, test_size=0.2)

In [8]:
#Train
indices = list(range(0, len(x_train), 1000))

indices.append(len(x_train))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_train, y_train, [1,2,3,4,5], indices[i], indices[i+1], 'topomaps_trigg_all_30', t='train')
    
#test
indices = list(range(0, len(x_test), 1000))

indices.append(len(x_test))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(x_test, y_test, [1,2,3,4,5], indices[i], indices[i+1], 'topomaps_trigg_all_30', t='test')
    
# Combine Train

combine_topomaps('topomaps_trigg_all_30/train/alpha', 'topomaps_trigg_all_30/train/beta', 'topomaps_trigg_all_30/train/gamma', 1, 'topomaps_trigg_all_30/train/combined')
combine_topomaps('topomaps_trigg_all_30/train/alpha', 'topomaps_trigg_all_30/train/beta', 'topomaps_trigg_all_30/train/gamma', 2, 'topomaps_trigg_all_30/train/combined')
combine_topomaps('topomaps_trigg_all_30/train/alpha', 'topomaps_trigg_all_30/train/beta', 'topomaps_trigg_all_30/train/gamma', 3, 'topomaps_trigg_all_30/train/combined')
combine_topomaps('topomaps_trigg_all_30/train/alpha', 'topomaps_trigg_all_30/train/beta', 'topomaps_trigg_all_30/train/gamma', 4, 'topomaps_trigg_all_30/train/combined')
combine_topomaps('topomaps_trigg_all_30/train/alpha', 'topomaps_trigg_all_30/train/beta', 'topomaps_trigg_all_30/train/gamma', 5, 'topomaps_trigg_all_30/train/combined')

# Combine Test

combine_topomaps('topomaps_trigg_all_30/test/alpha', 'topomaps_trigg_all_30/test/beta', 'topomaps_trigg_all_30/test/gamma', 1, 'topomaps_trigg_all_30/test/combined')
combine_topomaps('topomaps_trigg_all_30/test/alpha', 'topomaps_trigg_all_30/test/beta', 'topomaps_trigg_all_30/test/gamma', 2, 'topomaps_trigg_all_30/test/combined')
combine_topomaps('topomaps_trigg_all_30/test/alpha', 'topomaps_trigg_all_30/test/beta', 'topomaps_trigg_all_30/test/gamma', 3, 'topomaps_trigg_all_30/test/combined')
combine_topomaps('topomaps_trigg_all_30/test/alpha', 'topomaps_trigg_all_30/test/beta', 'topomaps_trigg_all_30/test/gamma', 4, 'topomaps_trigg_all_30/test/combined')
combine_topomaps('topomaps_trigg_all_30/test/alpha', 'topomaps_trigg_all_30/test/beta', 'topomaps_trigg_all_30/test/gamma', 5, 'topomaps_trigg_all_30/test/combined')

## Topomap Gen RT (Rate - 50)

In [17]:
data = pd.read_csv('data_RT_50/data_bp_RT_50.csv', header=None).values
labels = pd.read_csv('data_RT_50/labels_RT_50.csv', header=None).values
subjects = pd.read_csv('data_RT_50/subject_RT_50.csv', header=None)
unique_subs = np.unique(subjects)

scaler = MinMaxScaler()

X = scaler.fit_transform(data)
y = labels.astype(np.int16).flatten()

In [18]:
X_df = pd.DataFrame(X)
Y_df = pd.DataFrame(y)
X_df.insert(0, 'trial_no', ['bts_{}.png'.format(i) for i in np.arange(1, data.shape[0] + 1)])

subjects.insert(0, 'trial_no', ['bts_{}.png'.format(i) for i in np.arange(1, data.shape[0] + 1)])

Y_df.columns = ['labels']

subjects.columns = ['trial_no', 'subs']

In [19]:
X_df

,trial_no,0,1,2,3,4,5,6,7,8,...,182,183,184,185,186,187,188,189,190,191
0,bts_1.png,0.001153,0.000143,0.002179,0.000444,0.000940,0.012114,0.000105,0.000329,0.000305,...,0.000189,0.000156,0.000321,0.000416,0.000025,0.000229,0.000201,0.000197,0.000178,0.000216
1,bts_2.png,0.003383,0.000196,0.002839,0.000509,0.001543,0.006793,0.000028,0.000110,0.000115,...,0.000059,0.000071,0.000166,0.000278,0.000014,0.000092,0.000083,0.000086,0.000086,0.000104
2,bts_3.png,0.002366,0.000158,0.002680,0.000524,0.002084,0.005444,0.000105,0.000253,0.000251,...,0.000061,0.000070,0.000086,0.000292,0.000006,0.000090,0.000078,0.000078,0.000047,0.000082
3,bts_4.png,0.004592,0.000331,0.006751,0.000578,0.003266,0.013149,0.000046,0.000316,0.000242,...,0.000132,0.000131,0.000242,0.000581,0.000013,0.000175,0.000161,0.000167,0.000145,0.000176
4,bts_5.png,0.002612,0.000095,0.004595,0.000454,0.002104,0.009992,0.000200,0.000422,0.000429,...,0.000053,0.000082,0.000153,0.000275,0.000008,0.000078,0.000072,0.000096,0.000095,0.000113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62520,bts_62521.png,0.007808,0.002116,0.010399,0.005119,0.009993,0.012986,0.002681,0.006699,0.008175,...,0.013258,0.004086,0.021316,0.024282,0.001498,0.003931,0.004698,0.016370,0.064974,0.014012
62521,bts_62522.png,0.013677,0.002843,0.010258,0.011671,0.011672,0.007920,0.002430,0.009102,0.014027,...,0.027036,0.006001,0.036854,0.030037,0.001634,0.003823,0.004749,0.024205,0.103538,0.017674
62522,bts_62523.png,0.044592,0.009517,0.042001,0.042024,0.054755,0.012992,0.008534,0.036768,0.052456,...,0.039431,0.007398,0.032540,0.034019,0.002010,0.003674,0.004943,0.016704,0.100691,0.017916
62523,bts_62524.png,0.029406,0.007621,0.036215,0.018303,0.032274,0.044282,0.007879,0.014534,0.014792,...,0.214298,0.013244,0.044141,0.030387,0.001635,0.006629,0.009728,0.043237,0.106160,0.020900


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_df, Y_df, random_state=55, test_size=0.25, stratify=subjects['subs'])

In [21]:
pd.merge(X_test, subjects, on='trial_no').groupby('subs').count()

,trial_no,0,1,2,3,4,5,6,7,8,...,182,183,184,185,186,187,188,189,190,191
subs,,,,,,,,,,,,,,,,,,,,,
N1,250,250,250,250,250,250,250,250,250,250,...,250,250,250,250,250,250,250,250,250,250
N10,126,126,126,126,126,126,126,126,126,126,...,126,126,126,126,126,126,126,126,126,126
N11,250,250,250,250,250,250,250,250,250,250,...,250,250,250,250,250,250,250,250,250,250
N12,375,375,375,375,375,375,375,375,375,375,...,375,375,375,375,375,375,375,375,375,375
N13,375,375,375,375,375,375,375,375,375,375,...,375,375,375,375,375,375,375,375,375,375
N14,250,250,250,250,250,250,250,250,250,250,...,250,250,250,250,250,250,250,250,250,250
N15,250,250,250,250,250,250,250,250,250,250,...,250,250,250,250,250,250,250,250,250,250
N2,375,375,375,375,375,375,375,375,375,375,...,375,375,375,375,375,375,375,375,375,375
N3,375,375,375,375,375,375,375,375,375,375,...,375,375,375,375,375,375,375,375,375,375


In [22]:
X_test

,trial_no,0,1,2,3,4,5,6,7,8,...,182,183,184,185,186,187,188,189,190,191
49982,bts_49983.png,0.017987,0.003744,0.014288,0.011711,0.019667,0.002061,0.002121,0.007675,0.010853,...,0.000115,0.000254,0.000488,0.000907,0.000033,0.000120,0.000120,0.000268,0.000306,0.000386
4889,bts_4890.png,0.039747,0.002790,0.008442,0.005490,0.002737,0.003455,0.001321,0.003104,0.002659,...,0.000077,0.000067,0.000334,0.000192,0.000018,0.000073,0.000068,0.000115,0.000216,0.000301
12142,bts_12143.png,0.019563,0.004729,0.039984,0.016767,0.033696,0.007929,0.003601,0.012802,0.005551,...,0.000127,0.000495,0.001986,0.000932,0.000063,0.000858,0.000125,0.000586,0.001006,0.001009
33492,bts_33493.png,0.007475,0.000445,0.001554,0.001321,0.000901,0.001206,0.000389,0.000487,0.000591,...,0.000744,0.000117,0.000604,0.000305,0.000063,0.000129,0.000126,0.000151,0.000176,0.000231
26891,bts_26892.png,0.017857,0.004670,0.029656,0.014419,0.013647,0.001977,0.001018,0.004908,0.007210,...,0.000612,0.000201,0.001181,0.001030,0.000042,0.000186,0.000347,0.000255,0.004802,0.000772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49534,bts_49535.png,0.015231,0.002371,0.008042,0.011275,0.008456,0.004047,0.002799,0.007396,0.007110,...,0.000145,0.000062,0.000301,0.000502,0.000011,0.000030,0.000035,0.000066,0.000125,0.000133
44071,bts_44072.png,0.009904,0.001889,0.009195,0.005130,0.012107,0.002838,0.002468,0.007239,0.005738,...,0.000361,0.000177,0.001723,0.002951,0.000285,0.000242,0.000207,0.000252,0.001078,0.000261
34868,bts_34869.png,0.011846,0.004845,0.026283,0.015988,0.034730,0.013242,0.005857,0.003060,0.004762,...,0.000560,0.000628,0.001312,0.000762,0.000043,0.000401,0.000446,0.000665,0.000675,0.000799
37767,bts_37768.png,0.063714,0.001927,0.027473,0.043049,0.070132,0.003167,0.003865,0.021829,0.057476,...,0.001533,0.002730,0.006429,0.005975,0.000360,0.002115,0.001993,0.003680,0.004244,0.005017


In [23]:
subject_train = pd.merge(X_train, subjects, on='trial_no')[['subs', 'trial_no']]
# subject_val = x_val[['subs', 'trial_no']]
subject_test = pd.merge(X_test, subjects, on='trial_no')[['subs', 'trial_no']]

X_train = X_train.values[:,1:193]
X_test = X_test.values[:,1:193]

Y_train = y_train['labels'].values.flatten()
Y_test = y_test['labels'].values.flatten()

# subject_train = pd.read_csv("sub_train_rt_50.csv")
# subject_val = pd.read_csv("sub_val_rt_50.csv")
# subject_test = pd.read_csv("sub_test_rt_50.csv")

trial_train = subject_train['trial_no'].values
# trial_val = subject_val['trial_no'].values
trial_test = subject_test['trial_no'].values

In [24]:
subject_test.groupby('subs').count()

,trial_no
subs,
N1,250
N10,126
N11,250
N12,375
N13,375
N14,250
N15,250
N2,375
N3,375


In [13]:
np.savetxt("X_train.csv", X_train, delimiter=',')
np.savetxt("X_test.csv", X_test, delimiter=',')
np.savetxt("Y_train.csv", Y_train, delimiter=',')
np.savetxt("Y_test.csv", Y_test, delimiter=',')

In [25]:
subject_train.to_csv("sub_train_rt_50_h1.csv")
# subject_val.to_csv("sub_val_rt_50.csv")
subject_test.to_csv("sub_test_rt_50_h1.csv")

In [26]:
#Train
indices = list(range(0, len(X_train), 1000))

indices.append(len(X_train))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(X_train, Y_train, [0,1,3], trial_train, indices[i], indices[i+1], 'topomaps_RT_50_w_sub_h1', t='train')
    
#Validation
# indices = list(range(0, len(X_val), 1000))

# indices.append(len(X_val))

# for i in tqdm(range(len(indices) - 1)):
#     async_gen_topomap(X_val, Y_val, [0,1,3], trial_val, indices[i], indices[i+1], 'topomaps_RT_50_w_sub_h1', t='validation')
    
#test
indices = list(range(0, len(X_test), 1000))

indices.append(len(X_test))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(X_test, Y_test, [0,1,3], trial_test, indices[i], indices[i+1], 'topomaps_RT_50_w_sub_h1', t='test')
    
# Combine Train

combine_topomaps('topomaps_RT_50_w_sub_h1/train/alpha', 'topomaps_RT_50_w_sub_h1/train/beta', 'topomaps_RT_50_w_sub_h1/train/gamma', 0, 'topomaps_RT_50_w_sub_h1/train/combined')
combine_topomaps('topomaps_RT_50_w_sub_h1/train/alpha', 'topomaps_RT_50_w_sub_h1/train/beta', 'topomaps_RT_50_w_sub_h1/train/gamma', 1, 'topomaps_RT_50_w_sub_h1/train/combined')
combine_topomaps('topomaps_RT_50_w_sub_h1/train/alpha', 'topomaps_RT_50_w_sub_h1/train/beta', 'topomaps_RT_50_w_sub_h1/train/gamma', 3, 'topomaps_RT_50_w_sub_h1/train/combined')

# Combine Validation
# combine_topomaps('topomaps_RT_50_w_sub_h1/validation/alpha', 'topomaps_RT_50_w_sub_h1/validation/beta', 'topomaps_RT_50_w_sub_h1/validation/gamma', 0, 'topomaps_RT_50_w_sub_h1/validation/combined')
# combine_topomaps('topomaps_RT_50_w_sub_h1/validation/alpha', 'topomaps_RT_50_w_sub_h1/validation/beta', 'topomaps_RT_50_w_sub_h1/validation/gamma', 1, 'topomaps_RT_50_w_sub_h1/validation/combined')
# combine_topomaps('topomaps_RT_50_w_sub_h1/validation/alpha', 'topomaps_RT_50_w_sub_h1/validation/beta', 'topomaps_RT_50_w_sub_h1/validation/gamma', 3, 'topomaps_RT_50_w_sub_h1/validation/combined')

# # Combine Test

combine_topomaps('topomaps_RT_50_w_sub_h1/test/alpha', 'topomaps_RT_50_w_sub_h1/test/beta', 'topomaps_RT_50_w_sub_h1/test/gamma', 0, 'topomaps_RT_50_w_sub_h1/test/combined')
combine_topomaps('topomaps_RT_50_w_sub_h1/test/alpha', 'topomaps_RT_50_w_sub_h1/test/beta', 'topomaps_RT_50_w_sub_h1/test/gamma', 1, 'topomaps_RT_50_w_sub_h1/test/combined')
combine_topomaps('topomaps_RT_50_w_sub_h1/test/alpha', 'topomaps_RT_50_w_sub_h1/test/beta', 'topomaps_RT_50_w_sub_h1/test/gamma', 3, 'topomaps_RT_50_w_sub_h1/test/combined')

## Topomap Gen RT (Rate - 100)

In [3]:
data = pd.read_csv('data_RT_100/data_bp_RT_100.csv', header=None).values
labels = pd.read_csv('data_RT_100/labels_RT_100.csv', header=None).values
subjects = pd.read_csv('data_RT_100/subject_RT_100.csv', header=None).values.reshape((-1,))
unique_subs = np.unique(subjects)

scaler = MinMaxScaler()

X = scaler.fit_transform(data)
y = labels.astype(np.int16)

In [5]:
X_df = pd.DataFrame(X)
Y_df = pd.DataFrame(y)
Y_df.insert(0, 'trial_no', ['bts_{}.png'.format(i) for i in np.arange(1, data.shape[0] + 1)])
Y_df.insert(1, "subs", subjects)

Y_df.columns = ['trial_no', 'subs', 'labels']

In [6]:
Y_df

,trial_no,subs,labels
0,bts_1.png,N1,0
1,bts_2.png,N1,1
2,bts_3.png,N1,1
3,bts_4.png,N1,1
4,bts_5.png,N1,1
...,...,...,...
53139,bts_53140.png,T9,3
53140,bts_53141.png,T9,3
53141,bts_53142.png,T9,3
53142,bts_53143.png,T9,3


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_df, Y_df, random_state=33, test_size=0.2, stratify=Y_df['labels'])

In [8]:
subject_train = y_train[['subs', 'trial_no']]
# subject_val = x_val[['subs', 'trial_no']]
subject_test = y_test[['subs', 'trial_no']]

X_train = X_train.values
X_test = X_test.values

Y_train = y_train['labels'].values.flatten()
Y_test = y_test['labels'].values.flatten()

# subject_train = pd.read_csv("sub_train_rt_50.csv")
# subject_val = pd.read_csv("sub_val_rt_50.csv")
# subject_test = pd.read_csv("sub_test_rt_50.csv")

trial_train = subject_train['trial_no'].values
# trial_val = subject_val['trial_no'].values
trial_test = subject_test['trial_no'].values

In [9]:
subject_train.to_csv("sub_train_rt_100.csv")
# subject_val.to_csv("sub_val_rt_50.csv")
subject_test.to_csv("sub_test_rt_100.csv")

In [10]:
#Train
indices = list(range(0, len(X_train), 1000))

indices.append(len(X_train))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(X_train, Y_train, [0,1,3], trial_train, indices[i], indices[i+1], 'topomaps_RT_100_w_sub_2', t='train')
    
#Validation
# indices = list(range(0, len(X_val), 1000))

# indices.append(len(X_val))

# for i in tqdm(range(len(indices) - 1)):
#     async_gen_topomap(X_val, Y_val, [0,1,3], trial_val, indices[i], indices[i+1], 'topomaps_RT_100_w_sub_2', t='validation')
    
#test
indices = list(range(0, len(X_test), 1000))

indices.append(len(X_test))

for i in tqdm(range(len(indices) - 1)):
    async_gen_topomap(X_test, Y_test, [0,1,3], trial_test, indices[i], indices[i+1], 'topomaps_RT_100_w_sub_2', t='test')
    
# Combine Train

combine_topomaps('topomaps_RT_100_w_sub_2/train/alpha', 'topomaps_RT_100_w_sub_2/train/beta', 'topomaps_RT_100_w_sub_2/train/gamma', 0, 'topomaps_RT_100_w_sub_2/train/combined')
combine_topomaps('topomaps_RT_100_w_sub_2/train/alpha', 'topomaps_RT_100_w_sub_2/train/beta', 'topomaps_RT_100_w_sub_2/train/gamma', 1, 'topomaps_RT_100_w_sub_2/train/combined')
combine_topomaps('topomaps_RT_100_w_sub_2/train/alpha', 'topomaps_RT_100_w_sub_2/train/beta', 'topomaps_RT_100_w_sub_2/train/gamma', 3, 'topomaps_RT_100_w_sub_2/train/combined')

# Combine Validation
# combine_topomaps('topomaps_RT_100_w_sub_2/validation/alpha', 'topomaps_RT_100_w_sub_2/validation/beta', 'topomaps_RT_100_w_sub_2/validation/gamma', 0, 'topomaps_RT_100_w_sub_2/validation/combined')
# combine_topomaps('topomaps_RT_100_w_sub_2/validation/alpha', 'topomaps_RT_100_w_sub_2/validation/beta', 'topomaps_RT_100_w_sub_2/validation/gamma', 1, 'topomaps_RT_100_w_sub_2/validation/combined')
# combine_topomaps('topomaps_RT_100_w_sub_2/validation/alpha', 'topomaps_RT_100_w_sub_2/validation/beta', 'topomaps_RT_100_w_sub_2/validation/gamma', 3, 'topomaps_RT_100_w_sub_2/validation/combined')

# # Combine Test

combine_topomaps('topomaps_RT_100_w_sub_2/test/alpha', 'topomaps_RT_100_w_sub_2/test/beta', 'topomaps_RT_100_w_sub_2/test/gamma', 0, 'topomaps_RT_100_w_sub_2/test/combined')
combine_topomaps('topomaps_RT_100_w_sub_2/test/alpha', 'topomaps_RT_100_w_sub_2/test/beta', 'topomaps_RT_100_w_sub_2/test/gamma', 1, 'topomaps_RT_100_w_sub_2/test/combined')
combine_topomaps('topomaps_RT_100_w_sub_2/test/alpha', 'topomaps_RT_100_w_sub_2/test/beta', 'topomaps_RT_100_w_sub_2/test/gamma', 3, 'topomaps_RT_100_w_sub_2/test/combined')